## Python Project for Data Engineering

### You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

In [ ]:
import sqlite3
import numpy as np
import pandas as pd

In [22]:
db_name = 'Banks.db'
table_name = 'Largest_banks'
url = "https://en.wikipedia.org/wiki/List_of_largest_banks#By_market_capitalization"

In [3]:
df = pd.read_html(url)[2]

In [4]:
df

Bank name  \
                                  Bank name   
0                            JPMorgan Chase   
1                           Bank of America   
2   Industrial and Commercial Bank of China   
3                Agricultural Bank of China   
4                             Bank of China   
5                   China Construction Bank   
6                               Wells Fargo   
7                                      HSBC   
8                         Commonwealth Bank   
9                             Goldman Sachs   
10                     Royal Bank of Canada   
11                           Morgan Stanley   

   GlobalData (as of 30 September 2024)[4]                           \
                                      Rank Market cap (US$ billion)   
0                                      1.0                  599.931   
1                                      2.0                  307.900   
2                                      3.0                  303.543   
3                                      4.0                  232.836   
4                                      5.0                  209.295   
5                                      6.0                  192.715   
6                                      7.0                  192.279   
7                                      8.0                  163.544   
8                                      9.0                  156.639   
9                                     10.0                  156.356   
10                                     NaN                      NaN   
11                                     NaN                      NaN   

   Forbes India (as of 4 October 2024)[5]                           
                                     Rank Market cap (US$ billion)  
0                                     1.0                   583.91  
1                                     2.0                   304.56  
2                                     3.0                   288.06  
3                                     4.0                   231.00  
4                                     6.0                   187.79  
5                                     5.0                   197.15  
6                                     7.0                   187.13  
7                                    10.0                   159.35  
8                                     NaN                      NaN  
9                                     NaN                      NaN  
10                                    8.0                   173.86  
11                                    9.0                   169.17

In [5]:
df.columns

MultiIndex([(                              'Bank name', ...),
            ('GlobalData (as of 30 September 2024)[4]', ...),
            ('GlobalData (as of 30 September 2024)[4]', ...),
            ( 'Forbes India (as of 4 October 2024)[5]', ...),
            ( 'Forbes India (as of 4 October 2024)[5]', ...)],
           )

In [6]:
# Achata as colunas do MultiIndex, juntando o nível principal e o nível secundário com um espaço
df.columns = [' '.join(filter(None, col)).strip() for col in df.columns]

# Exibe as colunas para conferir os novos nomes após o achatamento
print(df.columns)


Index(['Bank name Bank name', 'GlobalData (as of 30 September 2024)[4] Rank',
       'GlobalData (as of 30 September 2024)[4] Market cap (US$ billion)',
       'Forbes India (as of 4 October 2024)[5] Rank',
       'Forbes India (as of 4 October 2024)[5] Market cap (US$ billion)'],
      dtype='object')


In [7]:
# Redefine manualmente os nomes das colunas para corrigir o erro de formatação
df.columns = [
    'Bank name',
    'GlobalData Rank',
    'GlobalData Market cap (US$ billion)',
    'Forbes India Rank',
    'Forbes India Market cap (US$ billion)'
]

# Seleciona as colunas 'Bank name' e 'GlobalData Market cap (US$ billion)'
df = df[['Bank name', 'GlobalData Market cap (US$ billion)']]
df


,Bank name,GlobalData Market cap (US$ billion)
0,JPMorgan Chase,599.931
1,Bank of America,307.900
2,Industrial and Commercial Bank of China,303.543
3,Agricultural Bank of China,232.836
4,Bank of China,209.295
5,China Construction Bank,192.715
6,Wells Fargo,192.279
7,HSBC,163.544
8,Commonwealth Bank,156.639
9,Goldman Sachs,156.356


In [8]:
df = df.head(10)
df

,Bank name,GlobalData Market cap (US$ billion)
0,JPMorgan Chase,599.931
1,Bank of America,307.900
2,Industrial and Commercial Bank of China,303.543
3,Agricultural Bank of China,232.836
4,Bank of China,209.295
5,China Construction Bank,192.715
6,Wells Fargo,192.279
7,HSBC,163.544
8,Commonwealth Bank,156.639
9,Goldman Sachs,156.356


In [ ]:
# Carregar as taxas de câmbio do arquivo CSV
exchange_rates = pd.read_csv('exchange_rate.csv')

# Criar um dicionário de taxas de câmbio
exchange_rate_dict = dict(zip(exchange_rates['Currency'], exchange_rates['Rate']))

# Selecionar as primeiras 10 linhas do DataFrame original
df = df.head(10).copy()

# Adicionar as colunas convertidas ao DataFrame usando .loc
df.loc[:, 'MC_GBP_Billion'] = [np.round(x * exchange_rate_dict['GBP'], 2) for x in df['GlobalData Market cap (US$ billion)']]
df.loc[:, 'MC_EUR_Billion'] = [np.round(x * exchange_rate_dict['EUR'], 2) for x in df['GlobalData Market cap (US$ billion)']]
df.loc[:, 'MC_INR_Billion'] = [np.round(x * exchange_rate_dict['INR'], 2) for x in df['GlobalData Market cap (US$ billion)']]

df

,Bank name,GlobalData Market cap (US$ billion),MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,599.931,479.94,557.94,49764.28
1,Bank of America,307.900,246.32,286.35,25540.30
2,Industrial and Commercial Bank of China,303.543,242.83,282.29,25178.89
3,Agricultural Bank of China,232.836,186.27,216.54,19313.75
4,Bank of China,209.295,167.44,194.64,17361.02
5,China Construction Bank,192.715,154.17,179.22,15985.71
6,Wells Fargo,192.279,153.82,178.82,15949.54
7,HSBC,163.544,130.84,152.10,13565.97
8,Commonwealth Bank,156.639,125.31,145.67,12993.21
9,Goldman Sachs,156.356,125.08,145.41,12969.73


In [10]:
df.rename(columns={'GlobalData Market cap (US$ billion)': 'MC_USD_Billion'}, inplace=True)
df

,Bank name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,599.931,479.94,557.94,49764.28
1,Bank of America,307.900,246.32,286.35,25540.30
2,Industrial and Commercial Bank of China,303.543,242.83,282.29,25178.89
3,Agricultural Bank of China,232.836,186.27,216.54,19313.75
4,Bank of China,209.295,167.44,194.64,17361.02
5,China Construction Bank,192.715,154.17,179.22,15985.71
6,Wells Fargo,192.279,153.82,178.82,15949.54
7,HSBC,163.544,130.84,152.10,13565.97
8,Commonwealth Bank,156.639,125.31,145.67,12993.21
9,Goldman Sachs,156.356,125.08,145.41,12969.73


In [18]:
df[['MC_USD_Billion','MC_GBP_Billion','MC_EUR_Billion','MC_INR_Billion']].astype(float).round(2)
df

,Bank name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,599.931,479.94,557.94,49764.28
1,Bank of America,307.900,246.32,286.35,25540.30
2,Industrial and Commercial Bank of China,303.543,242.83,282.29,25178.89
3,Agricultural Bank of China,232.836,186.27,216.54,19313.75
4,Bank of China,209.295,167.44,194.64,17361.02
5,China Construction Bank,192.715,154.17,179.22,15985.71
6,Wells Fargo,192.279,153.82,178.82,15949.54
7,HSBC,163.544,130.84,152.10,13565.97
8,Commonwealth Bank,156.639,125.31,145.67,12993.21
9,Goldman Sachs,156.356,125.08,145.41,12969.73


In [19]:
df['MC_EUR_Billion'][4]

194.64

In [20]:
df.to_csv('./largest_banks.csv', index=False)

In [23]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_name)

# Save the dataframe to the SQL database
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [27]:
conn = sqlite3.connect(db_name)
query = 'SELECT * FROM Largest_banks'
df = pd.read_sql_query(query, conn)
conn.close()
df

,Bank name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,599.931,479.94,557.94,49764.28
1,Bank of America,307.900,246.32,286.35,25540.30
2,Industrial and Commercial Bank of China,303.543,242.83,282.29,25178.89
3,Agricultural Bank of China,232.836,186.27,216.54,19313.75
4,Bank of China,209.295,167.44,194.64,17361.02
5,China Construction Bank,192.715,154.17,179.22,15985.71
6,Wells Fargo,192.279,153.82,178.82,15949.54
7,HSBC,163.544,130.84,152.10,13565.97
8,Commonwealth Bank,156.639,125.31,145.67,12993.21
9,Goldman Sachs,156.356,125.08,145.41,12969.73


In [28]:
conn = sqlite3.connect(db_name)
query = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
df = pd.read_sql_query(query, conn)
conn.close()
df

,AVG(MC_GBP_Billion)
0,201.202


In [32]:
conn = sqlite3.connect(db_name)
query = 'SELECT "Bank Name" from Largest_banks LIMIT 5'
df = pd.read_sql_query(query, conn)
conn.close()
df

,Bank name
0,JPMorgan Chase
1,Bank of America
2,Industrial and Commercial Bank of China
3,Agricultural Bank of China
4,Bank of China
